<a href="https://colab.research.google.com/github/fjadidi2001/AD_Prediction/blob/main/Denoise_May18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1: Setup Environment
from google.colab import drive
drive.mount('/content/drive')
!pip install torch torchvision opencv-python h5py scikit-image requests
!mkdir -p /content/drive/MyDrive/SSNet/data
!mkdir -p /content/drive/MyDrive/SSNet/output
!mkdir -p /content/drive/MyDrive/SSNet/saved_models/MyModel
!mkdir -p /content/drive/MyDrive/SSNet/logos
base_dir = '/content/drive/MyDrive/SSNet'

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully 

In [3]:
# Cell 2: Download and Preprocess BSR Dataset
import os
import cv2
import h5py
import numpy as np
import requests
import tarfile
from PIL import Image

base_dir = '/content/drive/MyDrive/SSNet'
os.makedirs(f'{base_dir}/data', exist_ok=True)
os.makedirs(f'{base_dir}/logos', exist_ok=True)

# Download BSDS500
url = 'https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/BSR/BSR_bsds500.tgz'
local_file = f'{base_dir}/BSR_bsds500.tgz'
print("Downloading BSDS500 dataset...")
try:
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(local_file, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Downloaded to {local_file}")
    else:
        raise Exception(f"Download failed. Status code: {response.status_code}")
except Exception as e:
    print(f"Error downloading dataset: {e}")
    print(f"Manually download from http://www.eecs.berkeley.edu/Research/Projects/CS/vision/bsds/BSR_bsds500.tgz")
    print(f"Upload to {local_file} using the next cell, then re-run this cell.")
    raise

# Extract dataset
try:
    with tarfile.open(local_file, 'r:gz') as tar:
        tar.extractall(base_dir)
    print("Extracted dataset.")
except Exception as e:
    print(f"Error extracting dataset: {e}")
    raise

# Process images into HDF5
train_dir = f'{base_dir}/BSR/BSDS500/data/images/train'
val_dir = f'{base_dir}/BSR/BSDS500/data/images/val'
test_dir = f'{base_dir}/BSR/BSDS500/data/images/test'

def create_hdf5(image_dir, output_path):
    images = []
    for img_name in os.listdir(image_dir):
        if img_name.endswith('.jpg'):
            img_path = os.path.join(image_dir, img_name)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Failed to load: {img_path}")
                continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img.astype(np.float32) / 255.0
            images.append(img)
    if not images:
        raise Exception(f"No images found in {image_dir}")
    with h5py.File(output_path, 'w') as f:
        f.create_dataset('data', data=np.array(images))
    print(f"Created HDF5: {output_path}")

try:
    create_hdf5(train_dir, f'{base_dir}/data/train_color_right.h5')
    create_hdf5(val_dir, f'{base_dir}/data/val_color_right.h5')
    create_hdf5(test_dir, f'{base_dir}/data/test_color_right.h5')
except Exception as e:
    print(f"Error creating HDF5 files: {e}")
    raise

# Create watermark logos
for i in range(1, 13):
    img = Image.new('RGBA', (100, 100), (255, 0, 0, 128))
    img.save(f'{base_dir}/logos/{i:02d}.png')
print("Created logos.")

Downloaded to /content/drive/MyDrive/SSNet/BSR_bsds500.tgz
Error extracting dataset: not a gzip file


ReadError: not a gzip file